<a href="https://colab.research.google.com/github/aman-17/actualwise/blob/main/simplified_chunking_actualwise_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract pdf2image
!apt-get install poppler-utils
!pip install --upgrade git+https://github.com/huggingface/transformers
!pip install openai==0.28

In [ ]:
!mkdir 'images1'
!mkdir 'images2'
!mkdir 'images3'
!mkdir 'chunk_deid1'
!mkdir 'chunk_deid2'
!mkdir 'chunk1'
!mkdir 'chunk2'
!mkdir 'chunk1_outputs'
!mkdir 'chunk2_outputs'
!mkdir 'merged_deid1'

In [ ]:
from pdf2image import convert_from_path
from PIL import Image
import pytesseract
import re
import os


oem_val = 3  # Example OEM value
psm_val = 6  # Example PSM value - 3, 4, 6? 1, 5, 7, 11, 12 (6 is best for Case 1)
custom_config = f'--oem {oem_val} --psm {psm_val}'

pdf_path = '/content/Case_study_1_CO.pdf'

output_folder = '/content/images2'
pdf_base_name = os.path.splitext(os.path.basename(pdf_path))[0]

output_file_name = f"{pdf_base_name}_oem{oem_val}_psm{psm_val}_pytesseract_poppler_dpi600.txt"
output_file_path = os.path.join(output_folder, output_file_name)

def clean_embedded_page_numbers(text):
    return re.sub(r'Page\s+\d+', '', text)

images = convert_from_path(pdf_path, 600)

with open(output_file_path, 'w', encoding='utf-8') as output_file:
    for i, image in enumerate(images):
        text = pytesseract.image_to_string(image, config=custom_config)
        cleaned_text = clean_embedded_page_numbers(text)
        # Inserting PDF page number
        output_file.write(f'PDF Page Number {i + 1}\n{cleaned_text}\n\n' + '-'*60 + '\n')

print(f"OCR text saved to {output_file_path}")

OCR text saved to /content/images2/Case_study_1_CO_oem3_psm6_pytesseract_poppler_dpi600.txt


In [ ]:
# Simplified chunking
import os
import re
from transformers import AutoTokenizer

# Setup
tokenizer = AutoTokenizer.from_pretrained("obi/deid_roberta_i2b2")
print("Tokenizer loaded.")

def tokenize_text_continuously(text, max_length):
    print("Tokenizing text and creating continuous chunks...")
    token_ids = tokenizer.encode(text, add_special_tokens=True)
    chunks = []
    token_start = 0
    while token_start < len(token_ids):
        token_end = min(token_start + max_length, len(token_ids))
        chunk_token_ids = token_ids[token_start:token_end]
        chunk = tokenizer.decode(chunk_token_ids, skip_special_tokens=True)
        chunks.append((chunk, token_start, token_end-1))
        token_start = token_end
    return chunks

def chunk_files(input_directory, output_directory, max_length=512):
    os.makedirs(output_directory, exist_ok=True)
    print(f"Processing files from {input_directory} to save chunks in {output_directory}")

    for filename in os.listdir(input_directory):
        if filename.endswith(".txt"):
            print(f"Processing file: {filename}")
            with open(os.path.join(input_directory, filename), 'r') as file:
                text = file.read()

            first_pass_chunks = tokenize_text_continuously(text, max_length)
            total_chunks = len(first_pass_chunks)
            max_digits_chunk = len(str(total_chunks))

            for i, (chunk, start_token, end_token) in enumerate(first_pass_chunks, start=1):
                chunk_num_padded = str(i).zfill(max_digits_chunk)
                chunk_filename = f"Chunk{chunk_num_padded}_Tokens{start_token}-{end_token}.txt"
                with open(os.path.join(output_directory, chunk_filename), 'w') as chunk_file:
                    chunk_file.write(chunk)
                    print(f"Saved chunk {chunk_filename}.")

# Example usage
input_directory = "/content/images2/"
output_directory = "/content/chunks/"
chunk_files(input_directory, output_directory)

tokenizer_config.json:   0%|          | 0.00/351 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (14078 > 512). Running this sequence through the model will result in indexing errors


Tokenizer loaded.
Processing files from /content/images2/ to save chunks in /content/chunks/
Processing file: Case_study_1_CO_oem3_psm6_pytesseract_poppler_dpi600.txt
Tokenizing text and creating continuous chunks...
Saved chunk Chunk01_Tokens0-511.txt.
Saved chunk Chunk02_Tokens512-1023.txt.
Saved chunk Chunk03_Tokens1024-1535.txt.
Saved chunk Chunk04_Tokens1536-2047.txt.
Saved chunk Chunk05_Tokens2048-2559.txt.
Saved chunk Chunk06_Tokens2560-3071.txt.
Saved chunk Chunk07_Tokens3072-3583.txt.
Saved chunk Chunk08_Tokens3584-4095.txt.
Saved chunk Chunk09_Tokens4096-4607.txt.
Saved chunk Chunk10_Tokens4608-5119.txt.
Saved chunk Chunk11_Tokens5120-5631.txt.
Saved chunk Chunk12_Tokens5632-6143.txt.
Saved chunk Chunk13_Tokens6144-6655.txt.
Saved chunk Chunk14_Tokens6656-7167.txt.
Saved chunk Chunk15_Tokens7168-7679.txt.
Saved chunk Chunk16_Tokens7680-8191.txt.
Saved chunk Chunk17_Tokens8192-8703.txt.
Saved chunk Chunk18_Tokens8704-9215.txt.
Saved chunk Chunk19_Tokens9216-9727.txt.
Saved chu

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the tokenizer and model from the pretrained "obi/deid_roberta_i2b2"
tokenizer = AutoTokenizer.from_pretrained("obi/deid_roberta_i2b2")
model = AutoModelForMaskedLM.from_pretrained("obi/deid_roberta_i2b2").to(device)
nlp = pipeline(task="ner", model="obi/deid_roberta_i2b2",device=0 if torch.cuda.is_available() else -1)

"""
tokenizer = AutoTokenizer.from_pretrained("nlpie/clinical-distilbert-i2b2-2010")
model = AutoModelForMaskedLM.from_pretrained("nlpie/clinical-distilbert-i2b2-2010").to(device)
nlp=pipeline(task="ner", model="obi/deid_bert_i2b2",device=0)
"""

# Define a function to deidentify text based on the NER results
def deidentify_text(text, ner_results):
    # Sort the entities based on their start position
    sorted_entities = sorted(ner_results, key=lambda x: x['start'])
    merged_entities = []

    # Merge overlapping entities
    for entity in sorted_entities:
        if merged_entities and entity['start'] <= merged_entities[-1]['end']:
            merged_entities[-1]['end'] = max(merged_entities[-1]['end'], entity['end'])
            merged_entities[-1]['entity'] = entity['entity']
        else:
            merged_entities.append(entity)

    # Replace the identified entities in the text with their entity type
    deidentified_text = text
    for entity in reversed(merged_entities):
        start = entity['start']
        end = entity['end']
        deidentified_text = deidentified_text[:start] + "[" + entity['entity'] + "]" + deidentified_text[end:]
    return deidentified_text

# Define the directory containing the text files
directory = "/content/chunks/"
# Define the directory to save the deidentified text files
new_directory = "/content/chunk_deid1"

# Loop through the text files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        # Open the text file and read the text
        with open(os.path.join(directory, filename), 'r') as file:
            text = file.read()
            # Perform NER on the text
            ner_results = nlp(text)
            # Deidentify the text
            deidentified_text = deidentify_text(text, ner_results)
        # Write the deidentified text to a new file
        with open(os.path.join(new_directory, "deidentified_" + filename), 'w') as file:
            file.write(deidentified_text)

config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at obi/deid_roberta_i2b2 and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
# Merge deid files
import os

# Define the directory containing the deidentified chunk files
input_directory = "/content/chunk_deid1/"
# Define the path for the output file
output_file_path = "/content/merged_deid1/merged_deidentified_text.txt"

# Get all filenames in the directory
filenames = os.listdir(input_directory)

# Sort the filenames simply by their natural order; this assumes the names are structured for correct sorting
sorted_filenames = sorted(filenames)

# Check if there are files to process
if not sorted_filenames:
    print("No files found in the directory.")
else:
    # Open the output file in write mode
    with open(output_file_path, 'w') as output_file:
        # Iterate over each sorted filename
        for filename in sorted_filenames:
            # Print the filename being processed
            # print(f"Including file in merge: {filename}")

            # Construct the full path to the file
            file_path = os.path.join(input_directory, filename)
            # Open and read the content of the file
            with open(file_path, 'r') as file:
                content = file.read()
                # Append the content to the output file
                output_file.write(content + "\n")  # Add a newline to separate each file's content

    print(f"All files have been successfully merged into {output_file_path}")



All files have been successfully merged into /content/merged_deid1/merged_deidentified_text.txt


In [ ]:
def split_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    half = len(lines) // 2
    with open('chunked_deid_file1.txt', 'w') as first_half_file:
        first_half_file.writelines(lines[:half])

    with open('chunked_deid_file2.txt', 'w') as second_half_file:
        second_half_file.writelines(lines[half:])

split_file('/content/merged_deid1/merged_deidentified_text.txt')


In [ ]:
import pandas as pd
import openai
from google.colab import userdata

# Ensure your OpenAI API key is correctly set here or through environment variables
openai.api_key = userdata.get('GPT_API')

# Read the file content
with open('/content/chunked_deid_file1.txt', 'r') as file:
    content1 = file.read()

df = pd.read_excel('./ChatGPT prompts v4.xlsx', sheet_name=1)
questions = df['Rafi\'s Qs'].tolist()

def get_gpt3_5_turbo_response(prompt):
    try:
        # Construct the messages parameter correctly as a list of dictionaries
        messages = [
            {"role": "system", "content": "You are a helpful medical record retriever."},
            {"role": "user", "content": prompt}
        ]

        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0125",
            messages=messages,
        )

        # Assuming the response structure matches expected output
        return response.choices[0].message['content'].strip() if response.choices else "No response"
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

with open('chunk1_answers.txt', 'w') as f:
    for question in questions:
        prompt = content1 + '\n Based on the above patient\'s medical record. Answer the following question. \n' + question
        response_text = get_gpt3_5_turbo_response(prompt)
        f.write(f"Question:\n{question}\nAnswer:\n{response_text}\n{'-'*32}\n")


In [ ]:
import pandas as pd
import openai
from google.colab import userdata

# Ensure your OpenAI API key is correctly set here or through environment variables
openai.api_key = userdata.get('GPT_API')

# Read the file content
with open('/content/chunked_deid_file2.txt', 'r') as file:
    content2 = file.read()

df = pd.read_excel('./ChatGPT prompts v4.xlsx', sheet_name=1)
questions = df['Rafi\'s Qs'].tolist()

def get_gpt3_5_turbo_response(prompt):
    try:
        # Construct the messages parameter correctly as a list of dictionaries
        messages = [
            {"role": "system", "content": "You are a helpful medical record retriever."},
            {"role": "user", "content": prompt}
        ]

        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0125",
            messages=messages,
        )

        # Assuming the response structure matches expected output
        return response.choices[0].message['content'].strip() if response.choices else "No response"
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

with open('chunk2_answers.txt', 'w') as f:
    for question in questions:
        prompt = content2 + '\n Based on the above patient\'s medical record. Answer the following question. \n' + question
        response_text = get_gpt3_5_turbo_response(prompt)
        f.write(f"Question:\n{question}\nAnswer:\n{response_text}\n{'-'*32}\n")


In [ ]:
# import openai
# from google.colab import userdata
# import pandas as pd

# # Load the data
# df = pd.read_excel('./ChatGPT prompts v4.xlsx', sheet_name=1)
# questions = df['Rafi\'s Qs'].tolist()

# # Read the file content
# with open('/content/merged_deid1/merged_deidentified_text.txt', 'r') as file:
#     content = file.read()

# # Convert the list into a string with each question numbered
# questions_str = '\n'.join([f'{i+1}. {q}' for i, q in enumerate(questions)])

# # Concatenate the strings
# prompt = content + '\n Based on the above patient\'s medical record. Answer the following questions? \n' + questions_str

# # print(prompt)


# # Ensure your OpenAI API key is correctly set here or through environment variables
# openai.api_key = userdata.get('GPT_API')

# def get_gpt3_5_turbo_response(prompt):
#     try:
#         # Construct the messages parameter correctly as a list of dictionaries
#         messages = [
#             {"role": "system", "content": "You are a helpful medical record retriever."},
#             {"role": "user", "content": prompt}
#         ]

#         response = openai.ChatCompletion.create(
#             model="gpt-3.5-turbo-0125",
#             # model = "gpt-4-0125-preview",
#             messages=messages,
#             # max_tokens=4096 ,  # Adjust based on the expected length of completion, ensuring total does not exceed 16385
#             # temperature=0.7,
#             # top_p=1,
#             # frequency_penalty=0,
#             # presence_penalty=0
#         )

#         # Assuming the response structure matches expected output
#         return response.choices[0].message['content'].strip() if response.choices else "No response"
#     except Exception as e:
#         print(f"An error occurred: {e}")
#         return None

# # Example usage
# # prompt = "What is the capital of France?"
# response_text = get_gpt3_5_turbo_response(prompt)
# print(response_text)


1. CRE: No
2. C-Auris: No
3. MRSA: No
4. C-DIFF: No
5. MDRO: No
6. COVID: No
7. A&OxNA
8. COVID vaccination: NA
9. COVID test: NA
10. No short-term care plan mentioned
11. No long-term care plan mentioned
12. Diagnoses: NA
13. • No past medical history (pmhx) mentioned
14. Mobility Aids: NA
15. Patient Monitoring & Safety Equipment: NA
16. Bed Equipment: NA
17. Specialized Medical Equipment: NA
18. Assistance: NA
19. IV: NA
20. Pressure ulcers: NA
21. Complex wounds: No
22. Wound Vac: No
23. Specialty Wound Equipment: No
24. No
25. NA
26. NA
27. Therapy: NA
28. Concerns: NA
29. Tuberculosis: No
30. VRE: No
31. HIV: No
32. ESBL: No
33. AIDS: No
34. ECG: No
35. PT/INR: No
36. eCardio: No
37. Bladder scan: No
38. Central line: No
39. Midline catheter: No
40. Implantable ports: No
41. JP/Penrose drain: No
42. Nephrostomy: No
43. Colostomy: No
44. Suprapubic Catheter: No
45. Dialysis Shunt: No
46. Dialysis Catheter: No
47. BiPAP/CPAP: No
48. Tracheostomy: No
49. Oxygen: No
50. Portable/Astr

In [ ]:
import os
import openai
import json
import pandas as pd
from google.colab import userdata
openai.api_key = userdata.get('GPT_API')


def process_file(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    df = pd.read_excel('./ChatGPT prompts v5.xlsx',sheet_name=1)
    questions = df['Rafi\'s Qs'].tolist()

    messages = [
        # {
        #     "role": "system",
        #     "content": "You are a helpful assistant."
        # },
    ]
    for question in questions:
        for chunk in chunks:
            if len(chunk) > 8192:
                print(f"Warning: chunk is too large ({len(chunk)} tokens)")
                continue
            messages.append({
                "role": "user",
                "content": chunk + question
            })
    return messages, questions

def get_responses(messages):
    responses = []
    for message in messages:
        response = openai.ChatCompletion.create(
            # engine="GUISE",
            deployment_id="test",
            messages=[message],
        )
        responses.append(response["choices"][0]["message"]["content"])
    return responses

def save_responses_to_file(responses, questions, output_file):
    with open(output_file, 'w') as f:
        for question, response in zip(questions, responses):
            f.write(f"{question}\n{response}\n{'-'*30}\n")

def process_files_in_folder(folder_path):
    files = [f for f in os.listdir(folder_path) if f.endswith(".txt")]
    for filename in files:
        file_path = os.path.join(folder_path, filename)
        output_file = os.path.join('/content/chunk1_outputs/', f'output_{filename}')
        messages, questions = process_file(file_path)
        responses = get_responses(messages)
        save_responses_to_file(responses, questions, output_file)

folder_path = './chunk_deid1/'
process_files_in_folder(folder_path)

In [ ]:
import os
import openai
import json
import pandas as pd
from google.colab import userdata
gpt4_api = userdata.get('gpt4_api')

openai.api_type = "azure"
openai.api_key = gpt4_api
openai.api_base = "https://test-gpt-4-ks.openai.azure.com"
openai.api_version = "2023-05-15"
openai.log = "debug"

def chunk_text(text, chunk_size=8192):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

def process_file(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    df = pd.read_excel('./ChatGPT prompts v5.xlsx',sheet_name=1)
    questions = df['Rafi\'s Qs'].tolist()
    chunks = chunk_text(text)
    messages = [
        # {
        #     "role": "system",
        #     "content": "You are a helpful assistant."
        # },
    ]
    for question in questions:
        for chunk in chunks:
            if len(chunk) > 8192:
                print(f"Warning: chunk is too large ({len(chunk)} tokens)")
                continue
            messages.append({
                "role": "user",
                "content": chunk + question
            })
    return messages, questions

def get_responses(messages):
    responses = []
    for message in messages:
        response = openai.ChatCompletion.create(
            # engine="GUISE",
            deployment_id="test",
            messages=[message],
        )
        responses.append(response["choices"][0]["message"]["content"])
    return responses

def save_responses_to_file(responses, questions, output_file):
    with open(output_file, 'w') as f:
        for question, response in zip(questions, responses):
            f.write(f"{question}\n{response}\n{'-'*30}\n")

def process_files_in_folder(folder_path):
    files = [f for f in os.listdir(folder_path) if f.endswith(".txt")]
    for filename in files:
        file_path = os.path.join(folder_path, filename)
        output_file = os.path.join('/content/chunk1_outputs/', f'output_{filename}')
        messages, questions = process_file(file_path)
        responses = get_responses(messages)
        save_responses_to_file(responses, questions, output_file)

folder_path = './chunk_deid1/'
process_files_in_folder(folder_path)

In [ ]:
import os
import openai
import json
import pandas as pd
from google.colab import userdata
gpt4_api = userdata.get('gpt4_api')

openai.api_type = "azure"
openai.api_key = gpt4_api
openai.api_base = "https://test-gpt-4-ks.openai.azure.com"
openai.api_version = "2023-05-15"
openai.log = "debug"

def chunk_text(text, chunk_size=8192):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

def process_file(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    chunks = chunk_text(text)
    messages = [
        # {
        #     "role": "system",
        #     "content": "You are a helpful assistant."
        # },
    ]
    for chunk in chunks:
        if len(chunk) > 8192:
            print(f"Warning: chunk is too large ({len(chunk)} tokens)")
            continue
        messages.append({
            "role": "user",
            "content": chunk + 'Summarize these question response statements, grouping similar items together, but include all page number references. Where statements include both \'Yes\' and \'No\', keep the \'Yes\' response with its page number references and omit the \'No\' statements. The goal is to aggregate the existing text, keeping it as concise as possible without adding any new text.'
        })
    return messages

def get_responses(messages):
    responses = []
    for message in messages:
        response = openai.ChatCompletion.create(
            # engine="GUISE",
            deployment_id="test",
            messages=[message],
        )
        responses.append(response["choices"][0]["message"]["content"])
    return responses

def save_responses_to_file(responses, output_file):
    with open(output_file, 'w') as f:
        for response in responses:
            f.write(f"{response}\n{'-'*30}\n")

def process_files_in_folder(folder_path):
    files = [f for f in os.listdir(folder_path) if f.endswith(".txt")]
    for filename in files:
        file_path = os.path.join(folder_path, filename)
        output_file = os.path.join('/content/', f'output_{filename}')
        messages = process_file(file_path)
        responses = get_responses(messages)
        save_responses_to_file(responses, output_file)

folder_path = '/content/'
process_files_in_folder(folder_path)